In [11]:
# Problem data
locations = ["Sellwood", "Hawthorne", "The Pearl", "Eastmoreland", \
                "St.Johns", "Alberta", "Nob Hill", "Belmont"]
roaster_cost = {"Sellwood":150000, "Hawthorne":100000, "The Pearl":250000, "Eastmoreland":120000, \
                "St.Johns":130000, "Alberta":110000, "Nob Hill":135000, "Belmont":180000}
roaster_capacity = {"Sellwood":12, "Hawthorne":18, "The Pearl":22, "Eastmoreland":13, \
                "St.Johns":14, "Alberta":10, "Nob Hill":17, "Belmont":12}
warehouse_cost = {"Sellwood":80000, "Hawthorne":90000, "The Pearl":120000, "Eastmoreland":90000, \
                  "St.Johns":85000, "Alberta":70000, "Nob Hill":85000, "Belmont":90000}
warehouse_size = {"Sellwood":8000, "Hawthorne":6000, "The Pearl":12000, "Eastmoreland":6000, \
                  "St.Johns":7000, "Alberta":9000, "Nob Hill":6000, "Belmont":9200}

min_total_sqft = 30000
min_roasting_capacity = 30

In [12]:
# Import PuLP Library
from pulp import *

In [13]:
# Create a Problem object
problem = LpProblem("Coffee Planning", LpMinimize)

In [14]:
# Create variables to indicate whether to build a Roaster at a given location
build_roaster = LpVariable.dicts("Build_Roaster", locations, 0, 1, LpInteger)

# Create variable to indicate whether to build a Warehouse at a given location
build_warehouse = LpVariable.dicts("Build_Warehouse", locations, 0, 1, LpInteger)

In [15]:
# Sum the cost of Roasters and Warehouses for the Objective Function
problem += \
    lpSum([ roaster_cost[l] * build_roaster[l] + warehouse_cost[l] * build_warehouse[l] \
            for l in locations])

In [16]:
# Total Roasting capacity must be greater than 30 tons
problem += lpSum([ roaster_capacity[l] * build_roaster[l] for l in locations]) >= min_roasting_capacity, "Min Roasting Capacity"

In [17]:
# Total Warehouse capacity must be greater than 30000 sq.ft.
problem += lpSum([ warehouse_size[l] * build_warehouse[l] for l in locations]) >= min_total_sqft, "Min Warehouse Capacity"

In [18]:
# Warehouse must exist where there is a roasting plant
for l in locations:
    problem += build_roaster[l] <= build_warehouse[l], f"Must Have Warehouse: {l}"

In [19]:
problem.solve()
# Check the status
print("Status:", LpStatus[problem.status])
for v in problem.variables():
    if v.varValue == 1:
        print(v.name, "=", v.varValue)
print("Total Cost = ", "${:,.2f}".format(value(problem.objective)))

Status: Optimal
Build_Roaster_Hawthorne = 1.0
Build_Roaster_St.Johns = 1.0
Build_Warehouse_Alberta = 1.0
Build_Warehouse_Hawthorne = 1.0
Build_Warehouse_Sellwood = 1.0
Build_Warehouse_St.Johns = 1.0
Total Cost =  $555,000.00


In [20]:
planned_roaster_capacity = sum([build_roaster[l].varValue * roaster_capacity[l]  for l in locations])
print(f"Roasting Capacity [tons]: {planned_roaster_capacity}")
planned_warehouse_capacity = sum([build_warehouse[l].varValue * warehouse_size[l] for l in locations])
print(f"Warehosue Capacity [sq.ft.]: {planned_warehouse_capacity}")

Roasting Capacity [tons]: 32.0
Warehosue Capacity [sq.ft.]: 30000.0
